In [1]:
from __future__ import print_function

import base64
import getopt
import itertools
import json
import math
import netrc
import os.path
import ssl
import sys
import time
from getpass import getpass

try:
    from urllib.parse import urlparse
    from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
    from urllib.error import HTTPError, URLError
except ImportError:
    from urlparse import urlparse
    from urllib2 import urlopen, Request, HTTPError, URLError, build_opener, HTTPCookieProcessor

In [2]:

short_name = 'GLAH01'
version = '033'
time_start = '2009-10-01T00:00:00Z'
time_end = '2009-10-02T23:59:59Z'
bounding_box = '-180,-86,180,86'
polygon = ''
filename_filter = ''
url_list = []

CMR_URL = 'https://cmr.earthdata.nasa.gov'
URS_URL = 'https://urs.earthdata.nasa.gov'
CMR_PAGE_SIZE = 2000
CMR_FILE_URL = ('{0}/search/granules.json?provider=NSIDC_ECS'
                '&sort_key[]=start_date&sort_key[]=producer_granule_id'
                '&scroll=true&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))


def get_username():
    username = ''

    # For Python 2/3 compatibility:
    try:
        do_input = raw_input  # noqa
    except NameError:
        do_input = input

    while not username:
        username = do_input('Earthdata username:zyjy0315 ')
    return username


def get_password():
    password = ''
    while not password:
        password = getpass('password:Stephen0315 ')
    return password


def get_credentials(url):
    """Get user credentials from .netrc or prompt for input."""
    credentials = None
    errprefix = ''
    try:
        info = netrc.netrc()
        username, account, password = info.authenticators(urlparse(URS_URL).hostname)
        errprefix = 'netrc error: '
    except Exception as e:
        if (not ('No such file' in str(e))):
            print('netrc error: {0}'.format(str(e)))
        username = None
        password = None

    while not credentials:
        if not username:
            username = get_username()
            password = get_password()
        credentials = '{0}:{1}'.format(username, password)
        credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')

        if url:
            try:
                req = Request(url)
                req.add_header('Authorization', 'Basic {0}'.format(credentials))
                opener = build_opener(HTTPCookieProcessor())
                opener.open(req)
            except HTTPError:
                print(errprefix + 'Incorrect username or password')
                errprefix = ''
                credentials = None
                username = None
                password = None

    return credentials


def build_version_query_params(version):
    desired_pad_length = 3
    if len(version) > desired_pad_length:
        print('Version string too long: "{0}"'.format(version))
        quit()

    version = str(int(version))  # Strip off any leading zeros
    query_params = ''

    while len(version) <= desired_pad_length:
        padded_version = version.zfill(desired_pad_length)
        query_params += '&version={0}'.format(padded_version)
        desired_pad_length -= 1
    return query_params


def filter_add_wildcards(filter):
    if not filter.startswith('*'):
        filter = '*' + filter
    if not filter.endswith('*'):
        filter = filter + '*'
    return filter


def build_filename_filter(filename_filter):
    filters = filename_filter.split(',')
    result = '&options[producer_granule_id][pattern]=true'
    for filter in filters:
        result += '&producer_granule_id[]=' + filter_add_wildcards(filter)
    return result


def build_cmr_query_url(short_name, version, time_start, time_end,
                        bounding_box=None, polygon=None,
                        filename_filter=None):
    params = '&short_name={0}'.format(short_name)
    params += build_version_query_params(version)
    params += '&temporal[]={0},{1}'.format(time_start, time_end)
    if polygon:
        params += '&polygon={0}'.format(polygon)
    elif bounding_box:
        params += '&bounding_box={0}'.format(bounding_box)
    if filename_filter:
        params += build_filename_filter(filename_filter)
    return CMR_FILE_URL + params


def get_speed(time_elapsed, chunk_size):
    if time_elapsed <= 0:
        return ''
    speed = chunk_size / time_elapsed
    if speed <= 0:
        speed = 1
    size_name = ('', 'k', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    i = int(math.floor(math.log(speed, 1000)))
    p = math.pow(1000, i)
    return '{0:.1f}{1}B/s'.format(speed / p, size_name[i])


def output_progress(count, total, status='', bar_len=60):
    if total <= 0:
        return
    fraction = min(max(count / float(total), 0), 1)
    filled_len = int(round(bar_len * fraction))
    percents = int(round(100.0 * fraction))
    bar = '=' * filled_len + ' ' * (bar_len - filled_len)
    fmt = '  [{0}] {1:3d}%  {2}   '.format(bar, percents, status)
    print('\b' * (len(fmt) + 4), end='')  # clears the line
    sys.stdout.write(fmt)
    sys.stdout.flush()


def cmr_read_in_chunks(file_object, chunk_size=1024 * 1024):
    """Read a file in chunks using a generator. Default chunk size: 1Mb."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


def cmr_download(urls, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None

    for index, url in enumerate(urls, start=1):
        if not credentials and urlparse(url).scheme == 'https':
            credentials = get_credentials(url)

        filename = url.split('/')[-1]
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        try:
            req = Request(url)
            if credentials:
                req.add_header('Authorization', 'Basic {0}'.format(credentials))
            opener = build_opener(HTTPCookieProcessor())
            response = opener.open(req)
            length = int(response.headers['content-length'])
            try:
                if not force and length == os.path.getsize(filename):
                    if not quiet:
                        print('  File exists, skipping')
                    continue
            except OSError:
                pass
            count = 0
            chunk_size = min(max(length, 1), 1024 * 1024)
            max_chunks = int(math.ceil(length / chunk_size))
            time_initial = time.time()
            with open(filename, 'wb') as out_file:
                for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                    out_file.write(data)
                    if not quiet:
                        count = count + 1
                        time_elapsed = time.time() - time_initial
                        download_speed = get_speed(time_elapsed, count * chunk_size)
                        output_progress(count, max_chunks, status=download_speed)
            if not quiet:
                print()
        except HTTPError as e:
            print('HTTP error {0}, {1}'.format(e.code, e.reason))
        except URLError as e:
            print('URL error: {0}'.format(e.reason))
        except IOError:
            raise


def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    entries = [e['links']
               for e in search_results['feed']['entry']
               if 'links' in e]
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()
    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue

        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        if filename in unique_filenames:
            # Exclude links with duplicate filenames (they would overwrite)
            continue
        unique_filenames.add(filename)

        urls.append(link['href'])

    return urls


def cmr_search(short_name, version, time_start, time_end,
               bounding_box='', polygon='', filename_filter='', quiet=False):
    """Perform a scrolling CMR query for files matching input criteria."""
    cmr_query_url = build_cmr_query_url(short_name=short_name, version=version,
                                        time_start=time_start, time_end=time_end,
                                        bounding_box=bounding_box,
                                        polygon=polygon, filename_filter=filename_filter)
    if not quiet:
        print('Querying for data:\n\t{0}\n'.format(cmr_query_url))

    cmr_scroll_id = None
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    urls = []
    hits = 0
    while True:
        req = Request(cmr_query_url)
        if cmr_scroll_id:
            req.add_header('cmr-scroll-id', cmr_scroll_id)
        response = urlopen(req, context=ctx)
        if not cmr_scroll_id:
            # Python 2 and 3 have different case for the http headers
            headers = {k.lower(): v for k, v in dict(response.info()).items()}
            cmr_scroll_id = headers['cmr-scroll-id']
            hits = int(headers['cmr-hits'])
            if not quiet:
                if hits > 0:
                    print('Found {0} matches.'.format(hits))
                else:
                    print('Found no matches.')
        search_page = response.read()
        search_page = json.loads(search_page.decode('utf-8'))
        url_scroll_results = cmr_filter_urls(search_page)
        if not url_scroll_results:
            break
        if not quiet and hits > CMR_PAGE_SIZE:
            print('.', end='')
            sys.stdout.flush()
        urls += url_scroll_results

    if not quiet and hits > CMR_PAGE_SIZE:
        print()
    return urls


def main(argv=None):
    global short_name, version, time_start, time_end, bounding_box, \
        polygon, filename_filter, url_list

    if argv is None:
        argv = sys.argv[1:]

    force = False
    quiet = False
    usage = 'usage: nsidc-download_***.py [--help, -h] [--force, -f] [--quiet, -q]'

    try:
        opts, args = getopt.getopt(argv, 'hfq', ['help', 'force', 'quiet'])
        for opt, _arg in opts:
            if opt in ('-f', '--force'):
                force = True
            elif opt in ('-q', '--quiet'):
                quiet = True
            elif opt in ('-h', '--help'):
                print(usage)
                sys.exit(0)
    except getopt.GetoptError as e:
        print(e.args[0])
        print(usage)
        sys.exit(1)

    # Supply some default search parameters, just for testing purposes.
    # These are only used if the parameters aren't filled in up above.
    if 'short_name' in short_name:
        short_name = 'ATL06'
        version = '003'
        time_start = '2018-10-14T00:00:00Z'
        time_end = '2021-01-08T21:48:13Z'
        bounding_box = ''
        polygon = ''
        filename_filter = '*ATL06_2020111121*'
        url_list = []

    try:
        if not url_list:
            url_list = cmr_search(short_name, version, time_start, time_end,
                                  bounding_box=bounding_box, polygon=polygon,
                                  filename_filter=filename_filter, quiet=quiet)

        cmr_download(url_list, force=force, quiet=quiet)
    except KeyboardInterrupt:
        quit()

In [3]:
if __name__ == '__main__':
    main()

Querying for data:
	https://cmr.earthdata.nasa.gov/search/granules.json?provider=NSIDC_ECS&sort_key[]=start_date&sort_key[]=producer_granule_id&scroll=true&page_size=2000&short_name=GLAH01&version=033&version=33&temporal[]=2009-10-01T00:00:00Z,2009-10-02T23:59:59Z&bounding_box=-180,-86,180,86

Found 121 matches.
Earthdata username:zyjy0315 zyjy0315
password:Stephen0315 ········
001/242: GLAH01_033_2131_001_1281_2_02_0001.H5
  [=====                                                       ]   8%  343.0kB/s    [=========                                                   ]  15%  372.4kB/s    [==============                                              ]  23%  378.1kB/s    [==================                                          ]  31%  391.0kB/s    [=======================                                     ]  38%  390.3kB/s    [============================                                ]  46%  383.9kB/s    [================================                            ]  54%  382.7kB/

  [====                                                        ]   6%  195.6kB/s    [========                                                    ]  12%  204.4kB/s    [===========                                                 ]  19%  166.0kB/s    [===============                                             ]  25%  175.9kB/s    [===================                                         ]  31%  180.8kB/s    [======================                                      ]  38%  185.4kB/s    [==========================                                  ]  44%  180.3kB/s    [==============================                              ]  50%  183.5kB/s    [==================================                          ]  56%  179.6kB/s    [======================================                      ]  62%  182.1kB/s    [=========================================                   ]  69%  183.8kB/s    [=============================================               ]  75%  185.8kB/s    [============

  [===                                                         ]   5%  470.4kB/s    [=====                                                       ]   9%  621.6kB/s    [========                                                    ]  14%  669.0kB/s    [===========                                                 ]  18%  685.2kB/s    [==============                                              ]  23%  683.3kB/s    [================                                            ]  27%  619.2kB/s    [===================                                         ]  32%  587.3kB/s    [======================                                      ]  36%  559.2kB/s    [=========================                                   ]  41%  539.1kB/s    [===========================                                 ]  45%  533.1kB/s    [==============================                              ]  50%  522.0kB/s    [=================================                           ]  55%  514.7kB/s    [============

  [=====                                                       ]   8%  75.6kB/s    [==========                                                  ]  17%  128.4kB/s    [===============                                             ]  25%  158.9kB/s    [====================                                        ]  33%  177.0kB/s    [=========================                                   ]  42%  188.3kB/s    [==============================                              ]  50%  169.1kB/s    [===================================                         ]  58%  177.7kB/s    [========================================                    ]  67%  191.7kB/s    [=============================================               ]  75%  203.0kB/s    [==================================================          ]  83%  212.4kB/s    [=======================================================     ]  92%  216.3kB/s    [============================================================] 100%  222.4kB/s   
064/242: GLAH01

  [====                                                        ]   7%  270.5kB/s    [=========                                                   ]  14%  303.3kB/s    [=============                                               ]  21%  339.1kB/s    [=================                                           ]  29%  377.2kB/s    [=====================                                       ]  36%  408.2kB/s    [==========================                                  ]  43%  432.8kB/s    [==============================                              ]  50%  437.5kB/s    [==================================                          ]  57%  426.3kB/s    [=======================================                     ]  64%  388.9kB/s    [===========================================                 ]  71%  336.3kB/s    [===============================================             ]  79%  338.7kB/s    [===================================================         ]  86%  342.3kB/s    [============

...

  [============================================================] 100%  584.5kB/s   
151/242: GLAH01_033_2131_001_1300_1_02_0001.H5
  [====                                                        ]   7%  213.6kB/s    [========                                                    ]  13%  198.3kB/s    [============                                                ]  20%  193.2kB/s    [================                                            ]  27%  218.9kB/s    [====================                                        ]  33%  191.4kB/s    [========================                                    ]  40%  186.5kB/s    [============================                                ]  47%  179.6kB/s    [================================                            ]  53%  170.7kB/s    [====================================                        ]  60%  171.9kB/s    [========================================                    ]  67%  150.4kB/s    [============================================

  [=====                                                       ]   8%  276.4kB/s    [=========                                                   ]  15%  264.7kB/s    [==============                                              ]  23%  238.5kB/s    [==================                                          ]  31%  219.0kB/s    [=======================                                     ]  38%  197.8kB/s    [============================                                ]  46%  203.4kB/s    [================================                            ]  54%  190.4kB/s    [=====================================                       ]  62%  190.9kB/s    [==========================================                  ]  69%  183.6kB/s    [==============================================              ]  77%  179.3kB/s    [===================================================         ]  85%  171.0kB/s    [=======================================================     ]  92%  171.9kB/s    [============

  [===                                                         ]   5%  229.2kB/s    [======                                                      ]  10%  321.9kB/s    [=========                                                   ]  14%  323.9kB/s    [===========                                                 ]  19%  202.2kB/s    [==============                                              ]  24%  168.8kB/s    [=================                                           ]  29%  147.4kB/s    [====================                                        ]  33%  157.8kB/s    [=======================                                     ]  38%  145.6kB/s    [==========================                                  ]  43%  156.3kB/s    [=============================                               ]  48%  161.7kB/s    [===============================                             ]  52%  163.9kB/s    [==================================                          ]  57%  152.9kB/s    [============

  [====                                                        ]   7%  260.6kB/s    [========                                                    ]  13%  203.4kB/s    [============                                                ]  20%  201.7kB/s    [================                                            ]  27%  207.5kB/s    [====================                                        ]  33%  215.9kB/s    [========================                                    ]  40%  227.2kB/s    [============================                                ]  47%  233.2kB/s    [================================                            ]  53%  240.7kB/s    [====================================                        ]  60%  254.2kB/s    [========================================                    ]  67%  267.2kB/s    [============================================                ]  73%  280.5kB/s    [================================================            ]  80%  291.8kB/s    [============

  [============================================================] 100%  965.6kB/s   
225/242: GLAH01_033_2131_001_1309_2_02_0001.H5
  [=====                                                       ]   8%  408.8kB/s    [=========                                                   ]  15%  552.3kB/s    [==============                                              ]  23%  592.6kB/s    [==================                                          ]  31%  586.0kB/s    [=======================                                     ]  38%  597.7kB/s    [============================                                ]  46%  613.8kB/s    [================================                            ]  54%  613.6kB/s    [=====================================                       ]  62%  604.8kB/s    [==========================================                  ]  69%  553.9kB/s    [==============================================              ]  77%  511.0kB/s    [============================================